- 댓글 크롤링 시간 줄이는 방법?
- 셀레니움 => bs4 페이지 소스는 가져올 수 있지만, 페이지 옵션 적용이 안 돼서 영어 번역 x  
- 모든 댓글 크롤링할 필요는 있을까? 시청자 성향 알기 위해서 우선 다수의 의견 반영이 돼야 함
  => 대댓글 성향 : 의견에 반대 or 공감 
        if 반대 : 원래 절대적인 반대 의견이 많지 않지만, 서로 댓글로 싸우면서 카운팅이 많아질 것(데이터 오류 생김), 욕설 난무 
        else :  ㅋㅋㅋㅋㅋ 위주의 댓글, 있으나 없으나 상관 없음. 부정적인 댓글이 아닐 경우 댓글이 많이 달리지 않고 추천이 달림.


In [31]:
# 옵션들
from selenium import webdriver  as wd
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.action_chains import ActionChains

options = wd.ChromeOptions() # 크롬 옵션 객체 생성
user_agent = "Mozilla/5.0 (Windows NT 4.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2049.0 Safari/537.36 "
options.add_argument('user-agent=' + user_agent)
#options.add_argument('headless') # headless 모드 설정
# options.add_argument("window-size=1920x1080") # 화면크기(전체화면)
options.add_argument("disable-gpu") 
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("--mute-audio") #mute
# options.add_argument("lang=ko_KR") # 가짜 플러그인 탑재
options.add_argument('--blink-settings=imagesEnabled=false') #브라우저에서 이미지 로딩을 하지 않습니다.
options.add_argument('incognito') #시크릿 모드의 브라우저가 실행됩니다.
options.add_argument("--start-maximized")
#options.add_argument('--kiosk') f11


# 속도 향상을 위한 옵션 해제
prefs = {
  "translate_whitelists": {"en":"ko"},
  "translate":{"enabled":"true"}
}
options.add_experimental_option("prefs", prefs) # 한국어로 자동 번역
prefs = {'profile.default_content_setting_values': {'cookies' : 2, 'images': 2, 'plugins' : 2, 'popups': 2, 'geolocation': 2, 'notifications' : 2, 'auto_select_certificate': 2, 'fullscreen' : 2, 'mouselock' : 2, 'mixed_script': 2, 'media_stream' : 2, 'media_stream_mic' : 2, 'media_stream_camera': 2, 'protocol_handlers' : 2, 'ppapi_broker' : 2, 'automatic_downloads': 2, 'midi_sysex' : 2, 'push_messaging' : 2, 'ssl_cert_decisions': 2, 'metro_switch_to_desktop' : 2, 'protected_media_identifier': 2, 'app_banner': 2, 'site_engagement' : 2, 'durable_storage' : 2}}   
options.add_experimental_option('prefs', prefs)

In [21]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

driver = webdriver.Chrome('C:/work/python/Asia_GAN/myproject/youtube/chromedriver.exe', options= options)
url = "https://www.youtube.com/c/NetflixKorea/videos"
driver.get(url)

# 스크롤 다운
time.sleep(5)
endk = 5
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1

# bs4 실행    
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

video_list0 = soup.find('div', {'id': 'contents'})
video_list2 = video_list0.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

base_url = 'http://www.youtube.com'
video_url = []

# 반복문을 실행시켜 비디오의 주소를 video_url에 넣는다.
for i in range(len(video_list2)):
    url = base_url+video_list2[i].find('a',{'id':'thumbnail'})['href']
    video_url.append(url)
    
print(video_url)

driver.close()

['http://www.youtube.com/watch?v=sXXjfLyoSKg', 'http://www.youtube.com/watch?v=xwHjaAqhVws', 'http://www.youtube.com/watch?v=GtT-XQrzmx0', 'http://www.youtube.com/watch?v=MeKOhbiphCQ', 'http://www.youtube.com/watch?v=zjX8ovfPUpY', 'http://www.youtube.com/watch?v=SJaFEM3pVRk', 'http://www.youtube.com/watch?v=Q5RDqTH_e8E', 'http://www.youtube.com/watch?v=K5S8e8YOzoY', 'http://www.youtube.com/watch?v=ulUH0Myr2AQ', 'http://www.youtube.com/watch?v=xDIHDmiqYBI', 'http://www.youtube.com/watch?v=hkPHYv59468', 'http://www.youtube.com/watch?v=qyklztSg0sE', 'http://www.youtube.com/watch?v=ikvX4-kHTw0', 'http://www.youtube.com/watch?v=5XFzUCN5il8', 'http://www.youtube.com/watch?v=Zm9HiGUni6c', 'http://www.youtube.com/watch?v=nSlTxOoOJS4', 'http://www.youtube.com/watch?v=irW2TmU-PXQ', 'http://www.youtube.com/watch?v=ekufuBs7izQ', 'http://www.youtube.com/watch?v=b96oSVw75lA', 'http://www.youtube.com/watch?v=-qCooQxYWws', 'http://www.youtube.com/watch?v=qbKvFIuMRIU', 'http://www.youtube.com/watch?v=2

In [32]:
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests
import re

def crawl_youtube_page_html_sources(video_url):
    html_sources = []

    for i in range(0,len(video_url)):
        driver = webdriver.Chrome('C:/work/python/Asia_GAN/myproject/youtube/chromedriver.exe', options= options)
        driver.get(video_url[i])
        body = driver.find_element_by_tag_name('body')
        time.sleep(0.5)

        #댓글 null 값 오류 방지 위해서 집어 넣음
        num_of_pagedowns = 1
        while num_of_pagedowns:
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(1)
            num_of_pagedowns -= 1

        last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

        while True:
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            driver.implicitly_wait(1.5)
            new_page_height = driver.execute_script("return document.documentElement.scrollHeight")

            if new_page_height == last_page_height:
                break
            last_page_height = new_page_height


        html_source = driver.page_source
        html_sources.append(html_source)
        print("OK")

        driver.quit()
    return html_sources

def get_user_IDs_and_comments(html_sources):
    my_dataframes = []
    titles = []
    for html in html_sources:
        
        soup = BeautifulSoup(html, 'lxml')
        
        youtube_title = soup.find('h1',{'class':'title style-scope ytd-video-primary-info-renderer'}).text.replace(',','')
        titles.append(youtube_title)
        
        #유튜브 닉네임, 유튜브 댓글       
        youtube_user_IDs = soup.select("div#header-author > h3 > #author-text > span")
        youtube_comments = soup.select("yt-formatted-string#content-text")
        
        str_youtube_userIDs = []
        str_youtube_comments = []
        
        
        for i in range(len(youtube_user_IDs)):
            str_tmp = str(youtube_user_IDs[i].text)
        #     print(str_tmp)
            str_tmp = str_tmp.replace('\n', '')
            str_tmp = str_tmp.replace('\t', '')
            str_tmp = str_tmp.replace('                ','')
            str_youtube_userIDs.append(str_tmp)

            str_tmp = str(youtube_comments[i].text) 
            str_tmp = str_tmp.replace('\n', '')
            str_tmp = str_tmp.replace('\t', '')
            str_tmp = str_tmp.replace('               ', '')

            str_youtube_comments.append(str_tmp)
            
        pd_data = {"ID":str_youtube_userIDs, "Comment":str_youtube_comments}

        youtube_pd = pd.DataFrame(pd_data)

        my_dataframes.append(youtube_pd)
        
    return my_dataframes, titles

def convert_csv_from_dataframe(titles, my_dataframes):
    for i in range(len(my_dataframes)):
        title = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…《\》]', '', titles[i])
        my_dataframes[i].to_csv("{}.csv".format(title))
        



In [33]:
video_url = ['https://www.youtube.com/watch?v=vDZLQo-N5Ts', 'https://www.youtube.com/watch?v=yQozPVwFGZA'] # 임시 url 실험
html_sources = crawl_youtube_page_html_sources(video_url)
my_dataframes, titles = get_user_IDs_and_comments(html_sources)
convert_csv_from_dataframe(titles, my_dataframes)

print('완료')

OK
OK
